# Additional End of week Exercise - week 2
## Assistant can call 2 tools to respond

* 2 tools
* No streaming
* learning - print tree structure of response, message, etc.
* 

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [102]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI  # import as class?
# import ollama
import gradio as gr


In [103]:
import openai  # import module
print(openai.__version__)


1.78.1


In [104]:
# constants

MODEL = 'gpt-4o-mini'
# MODEL_LLAMA = 'llama3.2'

In [105]:
# set up environment
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
openai = OpenAI()

API key looks good so far


In [106]:

system_message = (
    "You are a helpful assistant with access to two tools: "
    "1) 'count_letter_in_word' - to count how many times a letter appears in a word or the sentance. "
    "Use this when the user asks questions like 'How many R's are in Strawberry?', or " 
    "'how many E's are in this sentence?'. "
    "2) 'get_online_course_price' - to retrieve prices for online courses. "
    "Use this when the user asks about course prices like 'What is the price of AI Ethics course?'. "
    "Always pick the tool that best fits the user's question."
)

## Defined 2 tools:

In [107]:
# Let's start by making a useful function

def count_letter_in_word(word, letter):
    print(f"-> Tool: count_letter_in_word is called for counting letter in a word  => {word, letter}\n")
    counter = 0

    for c in word.lower():
        if c == letter.lower():
            counter += 1
    return counter

count_letter_in_word("how many a's in this sentance?", "A")

-> Tool: count_letter_in_word is called for counting letter in a word  => ("how many a's in this sentance?", 'A')



3

In [108]:
# There's a particular dictionary structure that's required to describe our function:
# has 2 parameters

count_function = {
    "name": "count_letter_in_word",
    "description": 
        "Count the number of a given letter in a specific word. "
        "Call this tool when the user asks questions like: "
        "'How many R's are in the word Strawberry?', "
        "'Count the letter A in banana', or "
        "'Number of T's in cat'.",
    "parameters": {
        "type": "object",
        "properties": {
            "word": {
                "type": "string",
                "description": "The word in which to count letters",
            },
            "letter": {
                "type": "string",
                "description": "The letter to count"
            }
            
        },
        "required": ["word", "letter"],
        "additionalProperties": False
    }
}

In [109]:
# Let's start by making a useful function

course_price = {"ai": "$119", "python": "$49", "machine learning": "$99", "llm": "$199"}

def get_online_course_price(online_course):
    print(f"-> Tool: get_online_course_price is called for course => {online_course}\n")
    
    course = online_course.lower()
    return course_price.get(course, "Unknown")

In [110]:
get_online_course_price("LLM")

-> Tool: get_online_course_price is called for course => LLM



'$199'

In [111]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_online_course_price",
    "description": 
        "Retrieve the price of an online course. "
        "Call this tool only when the user asks about course prices, "
        "such as 'What is the price of AI Ethics course?' or "
        "'How much does the Python course cost?'",
    "parameters": {
        "type": "object",
        "properties": {
            "online_course": {
                "type": "string",
                "description": 
                    "The online course that the customer wants to take."
                    "The exact name of the online course (e.g., 'AI', 'Python', 'LLM'). "
                    "Do NOT include extra words like 'course', 'class', or 'training'."
            },
        },
        "required": ["online_course"],
        "additionalProperties": False
    }
}

In [112]:
# List of tools API can call
tools = [
    {"type": "function", "function": count_function}, 
    {"type": "function", "function": price_function}
]

### history not really used, gradio can handle message cumulation

In [113]:
from pprint import pprint
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    # Convert to dict using the `.model_dump()` method (recommended for OpenAI SDK v1+)
    response_dict = response.model_dump()

    print("Full Response Tree:")
    print(json.dumps(response_dict, indent=2))

    print("\nAssistant Message Tree:")
    print(json.dumps(response_dict['choices'][0]['message'], indent=2))

    # assistant_message = response.choices[0].message.model_dump()
    # history.append({"role": "user", "content": message})  # Add user input to history

    
    
    if response.choices[0].finish_reason=="tool_calls":
        tool_messages = []
        for tool_call in response.choices[0].message.tool_calls:

            # tool_calls=[ChatCompletionMessageToolCall(id='call_mBxqr2fMPiYXppOtBnytGydo', 
            # function=Function(arguments='{"online_course":"LLM"}', 
            # name='get_online_course_price'), type='function')]))], 
            function_name = tool_call.function.name
            
            # arguments = {'online_course': 'LLM'}, a Python dictionary, from a json string
            # arguments = json.loads('{"online_course":"LLM"}')
            arguments = json.loads(tool_call.function.arguments)

            # Route to correct function
            if function_name == "count_letter_in_word":
                result = count_letter_in_word(**arguments)
                pprint(f"{function_name} result => {result}")
            elif function_name == "get_online_course_price":
                # if arguments = {'online_course': 'LLM'}, 
                # then **arguments is equivalent to calling the function with online_course="LLM".
                result = get_online_course_price(**arguments)
                pprint(f"{function_name} result => {result}")
                print(result)
                print()
            else:
                raise ValueError(f"Unknow tool: {function_name}")

            tool_response ={
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": json.dumps(result)
            }
            print(type(tool_response))
            pprint(f"tool_response => {tool_response}")
            print()
            tool_messages.append(tool_response)
            
        # # Add assistant's tool call message
        # history.append(assistant_message)
        # # Add tool responses
        # history.extend(tool_messages)
        
        
        # Convert to dict using the `.model_dump()` method (recommended for OpenAI SDK v1+)
        messages.append(response.choices[0].message.model_dump())

        print("Full message w/o tool_message Tree:")
        print(json.dumps(messages, indent=2))
        
        messages.extend(tool_messages)
        print("Full message w/ tool_message Tree:")
        print(type(messages))
        print(json.dumps(messages, indent=2))
        
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        # Convert to dict using the `.model_dump()` method (recommended for OpenAI SDK v1+)
        response_dict = response.model_dump()
        print("Full response after call tool Tree:")
        print(json.dumps(response_dict, indent=2))
        
    
    return response.choices[0].message.content

### Call gradio interfact and chat function, etc

In [114]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7893
* To create a public link, set `share=True` in `launch()`.


Full Response Tree:
{
  "id": "chatcmpl-BYJ8hSEXpSAJpSxp8XxEtOg9bOQN5",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Hello! How can I assist you today?",
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "audio": null,
        "function_call": null,
        "tool_calls": null
      }
    }
  ],
  "created": 1747516423,
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_54eb4bd693",
  "usage": {
    "completion_tokens": 10,
    "prompt_tokens": 330,
    "total_tokens": 340,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  }
}

Assistant Message Tree:
{
  "content": "Hello! How can

# streaming example but NOT working

In [ ]:
import gradio as gr
import openai

openai.api_key = api_key

system_message = "You are a helpful assistant."
MODEL = "gpt-3.5-turbo"

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        stream=True,
    )

    full_content = ""
    for chunk in stream:
        delta = chunk.choices[0].delta
        if "content" in delta:
            full_content += delta.content
            yield history + [{"role": "user", "content": message}, {"role": "assistant", "content": full_content}]
    
    # Do NOT use return here — just let the generator finish

gr.ChatInterface(fn=chat, type="messages").launch()
